In [8]:
from pathlib import Path

import eelbrain
import mne
from scipy.io import loadmat
import numpy as np



# Data locations
DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'
DATA_PREPROC = DATA_ROOT / "data_preprocessed"



In [9]:

for file in DATA_PREPROC.glob("*.mat"):

    SUBJECT = file.stem.split("_")[0]

    dst_dir = DATA_ROOT / 'envelopes' / SUBJECT
    dst_dir.mkdir(exist_ok=True, parents=True)
    attended_dst = dst_dir / f'{SUBJECT}_attended_envelope.pickle'
    unattended_dst = dst_dir / f'{SUBJECT}_unattended_envelope.pickle'
    padded_attended_dst = dst_dir / f'{SUBJECT}_padded_attended_envelope.pickle'
    padded_unattended_dst = dst_dir / f'{SUBJECT}_padded_unattended_envelope.pickle'
    if attended_dst.exists() and unattended_dst.exists() and padded_attended_dst.exists() and padded_unattended_dst.exists():
        print(f"Files for {SUBJECT} already exist, skipping.")
        continue


    print("Saving ~", SUBJECT, "...")

    mat = loadmat(file, squeeze_me=True, struct_as_record=False)
    data = mat['data']

    # --- collect all trials ---
    att_envelope_trials = []
    unatt_envelope_trials = []
    padded_att_envelope_trials = []
    padded_unatt_envelope_trials = []

    for att_envelope, unatt_envelope in zip(data.wavA, data.wavB):  # 60 trials
        # Create time axis
        time_axis = eelbrain.UTS(0, 1 / data.fsample.wavA, len(att_envelope))

        # Save the attended envelope as an NDVar
        att_ndvar = eelbrain.NDVar(att_envelope, dims=(time_axis,), name='attended')
        # Save the unattended envelope as an NDVar
        unatt_ndvar = eelbrain.NDVar(unatt_envelope, dims=(time_axis,), name='unattended')

        # Create padded versions
        padded_att_ndvar = eelbrain.pad(att_ndvar, tstart=-0.100, tstop=att_ndvar.time[-1] + 1, name='padded_attended')
        padded_unatt_ndvar = eelbrain.pad(unatt_ndvar, tstart=-0.100, tstop=unatt_ndvar.time[-1] + 1, name='padded_unattended')

        att_envelope_trials.append(att_ndvar)
        unatt_envelope_trials.append(unatt_ndvar)
        padded_att_envelope_trials.append(padded_att_ndvar)
        padded_unatt_envelope_trials.append(padded_unatt_ndvar)

    # --- concatenate trials ---
    att_envelope_concat = eelbrain.concatenate(att_envelope_trials)
    unatt_envelope_concat = eelbrain.concatenate(unatt_envelope_trials)
    padded_att_envelope_concat = eelbrain.concatenate(padded_att_envelope_trials)
    padded_unatt_envelope_concat = eelbrain.concatenate(padded_unatt_envelope_trials)

    # Save all 4 versions
    eelbrain.save.pickle(att_envelope_concat, attended_dst)
    eelbrain.save.pickle(unatt_envelope_concat, unattended_dst)
    eelbrain.save.pickle(padded_att_envelope_concat, padded_attended_dst)
    eelbrain.save.pickle(padded_unatt_envelope_concat, padded_unattended_dst)
    print(f"Saved attended, unattended, padded attended, and padded unattended envelopes for {SUBJECT}.")


Saving ~ S5 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S5.
Saving ~ S15 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S15.
Saving ~ S11 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S11.
Saving ~ S1 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S1.
Saving ~ S4 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S4.
Saving ~ S14 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S14.
Saving ~ S10 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S10.
Saving ~ S17 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S17.
Saving ~ S7 ...
Saved attended, unattended, padded attended, and padded unattended envelopes for S7.
Saving ~ S18 ...
Saved attended, unattended, padded attended, and padded unattend

In [10]:
# Sanity check: load one of the saved files and print its shape
att_ndvar = eelbrain.load.unpickle(DATA_ROOT / 'envelopes' / "S1" / "S1_attended_envelope.pickle")
unatt_ndvar = eelbrain.load.unpickle(DATA_ROOT / 'envelopes' / "S1" / "S1_unattended_envelope.pickle")
padded_att_ndvar = eelbrain.load.unpickle(DATA_ROOT / 'envelopes' / "S1" / "S1_padded_attended_envelope.pickle")
padded_unatt_ndvar = eelbrain.load.unpickle(DATA_ROOT / 'envelopes' / "S1" / "S1_padded_unattended_envelope.pickle")
print(att_ndvar, unatt_ndvar, padded_att_ndvar, padded_unatt_ndvar)

<NDVar 'attended': 192000 time> <NDVar 'unattended': 192000 time> <NDVar 'padded_attended': 196200 time> <NDVar 'padded_unattended': 196200 time>
